<a href="https://colab.research.google.com/github/Jeevesh8/arg_mining/blob/main/experiments/long_context_am.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install Dependencies

In [ ]:
%%capture
#if running on colab, install below 4
#!git clone https://github.com/Jeevesh8/arg_mining
#!pip install transformers
#!pip install seqeval datasets allennlp
#!pip install flax

#if connected to local runtime, run the next command too
#pip install bs4 tensorflow torch 



*   Update ``arg_mining/datasets/cmv_modes/configs.py`` as per your requirements, all experiments considered till now, set ``batch_size`` to 2, and all other variables with their default value.



In [4]:
#Run to ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Load Metric

In [5]:
from datasets import load_metric
metric = load_metric('seqeval')

### Define & Load Tokenizer, Model, Dataset

In [6]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
device

device(type='cuda', index=0)

In [8]:
model_version = 'allenai/longformer-base-4096'

In [9]:
%%capture
from transformers import LongformerTokenizer, AutoModel
tokenizer = LongformerTokenizer.from_pretrained(model_version)
transformer_model = AutoModel.from_pretrained(model_version).to(device)

In [10]:
with open('./Discourse_Markers.txt') as f:
    discourse_markers = [dm.strip() for dm in f.readlines()]

In [11]:
%%capture
from arg_mining.datasets.cmv_modes import load_dataset, data_config

tokenizer.add_tokens(data_config["special_tokens"])

transformer_model.resize_token_embeddings(len(tokenizer))

def get_datasets():
    train_dataset, valid_dataset, test_dataset = load_dataset(tokenizer=tokenizer,
                                                              train_sz=80,
                                                              test_sz=20,
                                                              mask_tokens=discourse_markers)
    return train_dataset, valid_dataset, test_dataset

### Define layers for a Linear-Chain-CRF

In [12]:
import torch.nn as nn

from allennlp.modules.conditional_random_field import ConditionalRandomField as crf

ac_dict = data_config["arg_components"]

allowed_transitions =([(ac_dict["B-C"], ac_dict["I-C"]), 
                       (ac_dict["B-P"], ac_dict["I-P"])] + 
                      [(ac_dict["I-C"], ac_dict[ct]) 
                        for ct in ["I-C", "B-C", "B-P", "O"]] +
                      [(ac_dict["I-P"], ac_dict[ct]) 
                        for ct in ["I-P", "B-C", "B-P", "O"]] +
                      [(ac_dict["O"], ac_dict[ct]) 
                        for ct in ["O", "B-C", "B-P"]])
                    
linear_layer = nn.Linear(transformer_model.config.hidden_size,
                         len(ac_dict)).to(device)

crf_layer = crf(num_tags=len(ac_dict),
                constraints=allowed_transitions,
                include_start_end_transitions=False).to(device)

cross_entropy_layer = nn.CrossEntropyLoss(weight=torch.log(torch.tensor([3.8087, 54.6686, 3.4161, 45.3662, 2.4729], 
                                                                        device=device)), reduction='none')

### Global Attention Mask Utility for Longformer

In [13]:
import numpy as np
from threading import Lock
MUTEX = Lock()

def get_global_attention_mask(tokenized_threads: np.ndarray) -> np.ndarray:
    """Returns an attention mask, with 1 where there are [USER{i}] tokens and 
    0 elsewhere.
    """
    mask = np.zeros_like(tokenized_threads)
    for user_token in ["UNU"]+[f"[USER{i}]" for i in range(data_config["max_users"])]:
        MUTEX.acquire()
        try:
            user_token_id = tokenizer.encode(user_token)[1:-1]
        finally:
            MUTEX.release()
        mask = np.where(tokenized_threads==user_token_id, 1, mask)
    return np.array(mask, dtype=bool)

### Loss and Prediction Function

In [14]:
from typing import Tuple

In [15]:
def compute(batch: Tuple[torch.Tensor, torch.Tensor, torch.Tensor],
            preds: bool=False, cross_entropy: bool=True):
    """
    Args:
        batch:  A tuple having tokenized thread of shape [batch_size, seq_len],
                component type labels of shape [batch_size, seq_len], and a global
                attention mask for Longformer, of the same shape.
        
        preds:  If True, returns a List(of batch_size size) of Tuples of form 
                (tag_sequence, viterbi_score) where the tag_sequence is the 
                viterbi-decoded sequence, for the corresponding sample in the batch.
        
        cross_entropy:  This argument will only be used if preds=False, i.e., if 
                        loss is being calculated. If True, then cross entropy loss
                        will also be added to the output loss.
    
    Returns:
        Either the predicted sequences with their scores for each element in the batch
        (if preds is True), or the loss value summed over all elements of the batch
        (if preds is False).
    """
    tokenized_threads, comp_type_labels, global_attention_mask = batch
    
    pad_mask = torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0)
    
    logits = linear_layer(transformer_model(input_ids=tokenized_threads,
                               attention_mask=pad_mask,
                               global_attention_mask=global_attention_mask).last_hidden_state)
    
    if preds:
        return crf_layer.viterbi_tags(logits, pad_mask)
    
    log_likelihood = crf_layer(logits, comp_type_labels, pad_mask)
    
    if cross_entropy:
        logits = logits.reshape(-1, logits.shape[-1])
        
        pad_mask, comp_type_labels = pad_mask.reshape(-1), comp_type_labels.reshape(-1)
        
        ce_loss = torch.sum(pad_mask*cross_entropy_layer(logits, comp_type_labels))
        
        return ce_loss - log_likelihood

    return -log_likelihood

### Define optimizer

In [16]:
from itertools import chain

import torch.optim as optim

optimizer = optim.Adam(params = chain(transformer_model.parameters(),
                                      linear_layer.parameters(),
                                      crf_layer.parameters()),
                       lr = 2e-5,)

### Training And Evaluation Loops

In [17]:
def train(dataset):
    accumulate_over = 4
    
    optimizer.zero_grad()

    for i, (tokenized_threads, masked_threads, comp_type_labels, _ ) in enumerate(dataset):
        global_attention_mask = torch.tensor(get_global_attention_mask(tokenized_threads), 
                                             device=device, dtype=torch.int32)
        
        #Remove Device Axis and cast to PyTorch tensor
        tokenized_threads = torch.tensor(np.squeeze(tokenized_threads, axis=0), 
                                        device=device)
        masked_threads = torch.tensor(np.squeeze(masked_threads, axis=0), 
                                     device=device)
        comp_type_labels = torch.tensor(np.squeeze(comp_type_labels, axis=0), 
                                        device=device, dtype=torch.long)

        loss = compute((masked_threads, 
                        comp_type_labels, 
                        global_attention_mask))/data_config["batch_size"]
        
        print("Loss: ", loss)
        loss.backward()
        
        if i%accumulate_over==accumulate_over-1:
            optimizer.step()
            optimizer.zero_grad()
    
    optimizer.step()

In [18]:
def evaluate(dataset, metric):
    
    int_to_labels = {v:k for k, v in ac_dict.items()}
    
    for tokenized_threads, masked_threads, comp_type_labels, _ in dataset:
        
        global_attention_mask = torch.tensor(get_global_attention_mask(tokenized_threads), 
                                             device=device)
        
        #Remove Device Axis and cast to PyTorch tensor
        tokenized_threads = torch.tensor(np.squeeze(tokenized_threads, axis=0), 
                                        device=device)
        masked_threads = torch.tensor(np.squeeze(masked_threads, axis=0), 
                                     device=device)
        comp_type_labels = torch.tensor(np.squeeze(comp_type_labels, axis=0), 
                                        device=device)

        preds = compute((masked_threads,
                         comp_type_labels,
                         global_attention_mask),
                        preds=True)
        
        lengths = torch.sum(torch.where(tokenized_threads!=tokenizer.pad_token_id, 1, 0), 
                            axis=-1)
        
        preds = [ [int_to_labels[pred] for pred in pred[0][:lengths[i]]]
                  for i, pred in enumerate(preds)
                ]
        
        refs = [ [int_to_labels[ref] for ref in labels[:lengths[i]]]
                 for i, labels in enumerate(comp_type_labels.cpu().tolist())
               ]
        
        metric.add_batch(predictions=preds, 
                         references=refs)
    
    print(metric.compute())

### Final Training

In [19]:
n_epochs = 35

In [20]:
for epoch in range(n_epochs):
    print(f"------------EPOCH {epoch+1}---------------")
    train_dataset, _, test_dataset = get_datasets()
    train(train_dataset)
    evaluate(test_dataset, metric)

------------EPOCH 1---------------


fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.
2021-08-25 09:38:18.141123: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/extras/CUPTI/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-08-25 09:38:18.141929: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Loss:  tensor(16992.0215, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(17061.4336, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(17101.1875, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(17003.9492, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(16729.6367, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(16692.6406, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(16562.7461, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(16325.4580, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(15628.1758, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(15663.8633, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(15760.4961, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(15693.3105, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(14728.2812, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(14684.8145, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(14926.4619, device='cuda:0', grad_fn=<DivBackwar

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(3039.3184, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2567.7722, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3261.6062, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2839.3145, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2130.6138, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2099., device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2451.1877, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2620.4292, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1803.1233, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(1924.6633, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2110.2139, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3006.3013, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(2465.2861, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3563.8293, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(3756.8403, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(

...

Loss:  tensor(80.0171, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(47.9101, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(110.6246, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(51.6392, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(56.2038, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(49.5531, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(38.5965, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(44.9701, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(20.8578, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(26.0513, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(44.1838, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(79.2387, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(63.6329, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(83.6263, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(111.0328, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(83.9348, device='cuda:0'

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(70.3565, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(44.9309, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(104.2675, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(47.6244, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(50.7763, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(46.0933, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(38.4234, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(41.0651, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(17.5519, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(21.9630, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(41.1081, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(69.2878, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(50.6910, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(76.0012, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(104.3971, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(67.6964, device='cuda:0'

fatal: destination path 'change-my-view-modes' already exists and is not an empty directory.


Loss:  tensor(60.4560, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(37.4781, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(100.6834, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(42.2467, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(39.4425, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(42.7611, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(34.1527, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(33.9432, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(16.3058, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(20.8814, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(36.2624, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(62.2585, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(48.6947, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(69.8019, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(87.3918, device='cuda:0', grad_fn=<DivBackward0>)
Loss:  tensor(63.0988, device='cuda:0',